In [8]:
import math
import numpy as np
import random
from random import randint
from gurobipy import *
#import pandas as pd
from random import seed
import matplotlib.pyplot as plt
import geopy.distance

In [12]:
#path = r"C:/Users/Devika Kabe/Documents/Model_brainstorming/"
path = "/home/dkabe/Model_brainstorming/"

In [13]:
f = open(path + 'Input_Data/product_distributions.txt', 'r')
text = f.read()
f.close()
distributions = text.split('\n')

In [14]:
for i in distributions:
    exec(i.lstrip().rstrip())

In [15]:
distribution_mat = np.array([[p1_mean, p1_sd],
                             [p2_mean, p2_sd],
                             [p3_mean, p3_sd],
                             [p4_mean, p4_sd]])

In [16]:
distribution_mat

array([[2487.        ,   70.71067812],
       [2329.        ,   42.42640687],
       [2539.        ,   14.14213562],
       [2302.        ,   56.56854249]])

## 2 Products, 2 Outsourced Suppliers

In [30]:
Products = 2
Outsourced = 2

In [31]:
## 2 MPs, 3 DCs, 4 MZs
## 2 Products, 2 Outsourced Facilities 

Manufacturing_plants = 2
Distribution = 3
num_Scenarios = (2**Manufacturing_plants)*(2**Distribution) 
Market = 1

# Cost of opening
f_i = [7500, 12500]
f_j = [7500, 12500, 17500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000

Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000

Capacities_l = np.random.randint(400, 500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [11]:
lost_sales

array([[22.08511002, 23.60162247]])

In [12]:
Supplier_cost + T_O_MZ

array([[[21.19525402, 21.6943816 ],
        [21.52347949, 21.34983892]],

       [[23.87372777, 24.54044572],
        [23.85764482, 25.22020219]]])

In [11]:
T_O_DC

array([[[1.5488135 , 1.71518937, 1.60276338],
        [1.54488318, 1.4236548 , 1.64589411]],

       [[1.43758721, 1.891773  , 1.96366276],
        [1.38344152, 1.79172504, 1.52889492]]])

In [12]:
# save files 
np.savetxt(path + 'Input_Data/Instance_1/OpenMP_1.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_1/OpenDC_1.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_1/Manufacturing_1.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_1/TransMPDC_1.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_1/TransDCMZ_1.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesMP_1.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesDC_1.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesOutsource_1.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_1/SupplierCost_1.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_1/TransSupplierDC_1.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_1/TransSupplierMZ_1.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_1/Volume_1.txt', volume)
np.savetxt(path + 'Input_Data/Instance_1/LostSales_1.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_1/Demand_1.txt', demand.reshape((num_Scenarios*Products, Market)))

In [13]:
## 3 MPs, 4 DCs, 2 MZs

num_Scenarios = 128

Manufacturing_plants = 3
Distribution = 4
Market = 2
Outsourced = 2

# Cost of opening
f_i = [7500, 12500, 17500]
f_j = [7500, 12500, 17500, 7500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_l = np.random.randint(800, 1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [14]:
np.sum(demand[1])

9784.0

In [15]:
np.savetxt(path + 'Input_Data/Instance_2/OpenMP_2.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_2/OpenDC_2.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_2/Manufacturing_2.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_2/TransMPDC_2.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_2/TransDCMZ_2.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesMP_2.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesDC_2.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesOutsource_2.txt', Capacities_l)
np.savetxt(path +'Input_Data/Instance_2/SupplierCost_2.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_2/TransSupplierDC_2.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_2/TransSupplierMZ_2.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_2/Volume_2.txt', volume)
np.savetxt(path + 'Input_Data/Instance_2/LostSales_2.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_2/Demand_2.txt', demand.reshape((num_Scenarios*Products, Market)))

In [16]:
demand[:21,:,:1]

array([[[2612.],
        [2346.]],

       [[2619.],
        [2288.]],

       [[2480.],
        [2346.]],

       [[2541.],
        [2334.]],

       [[2593.],
        [2320.]],

       [[2306.],
        [2357.]],

       [[2647.],
        [2267.]],

       [[2595.],
        [2391.]],

       [[2424.],
        [2245.]],

       [[2574.],
        [2380.]],

       [[2413.],
        [2269.]],

       [[2451.],
        [2310.]],

       [[2373.],
        [2320.]],

       [[2451.],
        [2279.]],

       [[2492.],
        [2342.]],

       [[2439.],
        [2314.]],

       [[2500.],
        [2312.]],

       [[2423.],
        [2331.]],

       [[2568.],
        [2277.]],

       [[2425.],
        [2304.]],

       [[2405.],
        [2367.]]])

In [17]:
Outsourced

2

In [18]:
## 4 MPs, 6 DCs, 3 MZs

num_Scenarios = 200
Manufacturing_plants = 4
Distribution = 6
Market = 3
Products = 2
Outsourced = 2
# Cost of opening
f_i = [7500, 12500, 17500, 7500]
f_j = [7500, 12500, 17500, 7500, 12500, 17500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_i[3] = 5000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_j[4] = 10000
Capacities_j[5] = 15000
Capacities_l = np.random.randint(1200, 1500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [19]:
lost_sales

array([[22.08511002, 23.60162247],
       [20.00057187, 21.51166286],
       [20.73377945, 20.46169297]])

In [20]:
Supplier_cost

array([[[18.64644051, 19.1455681 ],
        [18.80829013, 18.63464955]],

       [[21.2709644 , 21.93768234],
        [21.31276163, 22.675319  ]]])

In [21]:
np.savetxt(path + 'Input_Data/Instance_3/OpenMP_3.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_3/OpenDC_3.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_3/Manufacturing_3.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_3/TransMPDC_3.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_3/TransDCMZ_3.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesMP_3.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesDC_3.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesOutsource_3.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_3/SupplierCost_3.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_3/TransSupplierDC_3.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_3/TransSupplierMZ_3.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_3/Volume_3.txt', volume)
np.savetxt(path + 'Input_Data/Instance_3/LostSales_3.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_3/Demand_3.txt', demand.reshape((num_Scenarios*Products, Market)))

In [22]:
## 6 MPs, 8 DCs, 5 MZs

num_Scenarios = 200
Manufacturing_plants = 6
Distribution = 8
Market = 5

# Cost of opening
f_i = [7500, 12500, 17500, 7500, 12500, 17500]
f_j = [7500, 12500, 17500, 7500, 12500, 17500, 7500, 12500]
# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_i[3] = 5000
Capacities_i[4] = 10000
Capacities_i[5] = 15000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_j[4] = 10000
Capacities_j[5] = 15000
Capacities_j[6] = 5000
Capacities_j[7] = 10000
Capacities_l = np.random.randint(2000, 2500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [23]:
np.sum(demand[1])

24339.0

In [24]:
demand

array([[[2612., 2556., 2619., 2554., 2480.],
        [2346., 2424., 2288., 2323., 2346.]],

       [[2497., 2541., 2518., 2593., 2509.],
        [2391., 2334., 2343., 2320., 2293.]],

       [[2306., 2548., 2647., 2490., 2595.],
        [2357., 2298., 2267., 2321., 2391.]],

       ...,

       [[2577., 2488., 2463., 2542., 2430.],
        [2299., 2343., 2328., 2347., 2282.]],

       [[2431., 2476., 2445., 2517., 2398.],
        [2329., 2294., 2265., 2327., 2330.]],

       [[2582., 2580., 2501., 2586., 2406.],
        [2298., 2315., 2333., 2336., 2273.]]])

In [25]:
np.savetxt(path + 'Input_Data/Instance_4/OpenMP_4.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_4/OpenDC_4.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_4/Manufacturing_4.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_4/TransMPDC_4.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_4/TransDCMZ_4.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesMP_4.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesDC_4.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesOutsource_4.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_4/SupplierCost_4.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_4/TransSupplierDC_4.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_4/TransSupplierMZ_4.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_4/Volume_4.txt', volume)
np.savetxt(path + 'Input_Data/Instance_4/LostSales_4.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_4/Demand_4.txt', demand.reshape((num_Scenarios*Products, Market)))

In [17]:
with open (path + "Input_Data/market_locations.txt") as f:
    market_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [18]:
with open (path + "Input_Data/DC_locations.txt") as f:
    DC_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [19]:
with open(path + "Input_Data/outsource_locations.txt") as f:
    outsource_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [20]:
distances = []
for loc1 in range(len(DC_locations)):
    temp_dist = []
    for loc2 in range(len(market_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(DC_locations[loc1], market_locations[loc2]))[:-3]))
    distances.append(temp_dist)

In [25]:
distances_o_j = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(DC_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], DC_locations[loc2]))[:-3]))
    distances_o_j.append(temp_dist)


In [23]:
distances_o_k = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(market_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], market_locations[loc2]))[:-3]))
    distances_o_k.append(temp_dist)

In [34]:
## 6 MPs, 4 DCs, 29 MZs
Products = 3
Outsourced = 3
num_Scenarios = 200
Manufacturing_plants = 6
Distribution = 3
Market = 29
f_i = [None]*Manufacturing_plants
f_j = [None]*Distribution

np.random.seed(0)
Transportation_i_j = np.random.uniform(0, 0.1, (Products, Manufacturing_plants, Distribution))
Transportation_j_k = np.zeros((Products, Distribution, Market))
for m in range(Products):
    for j in range(Distribution):
        for k in range(Market):
            Transportation_j_k[m][j][k] = distances[j][k]/1000

T_O_DC = np.zeros((Products, Outsourced, Distribution))
for m in range(Products):
    for o in range(Outsourced):
        for j in range(Distribution):
            T_O_DC[m][o][j] = distances_o_j[o][j]/1000

T_O_MZ = np.zeros((Products, Outsourced, Market))
for m in range(Products):
    for o in range(Outsourced):
        for k in range(Market):
            T_O_MZ[m][o][k] = distances_o_k[o][k]/1000

levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0][0][0] = 10
Supplier_cost[0][1][1] = 5
Supplier_cost[0][2][2] = 7

Supplier_cost[1][0][0] = 11
Supplier_cost[1][1][1] = 6
Supplier_cost[1][2][2] = 8


In [35]:
Supplier_cost

array([[[10.,  0.,  0.],
        [ 0.,  5.,  0.],
        [ 0.,  0.,  7.]],

       [[11.,  0.,  0.],
        [ 0.,  6.,  0.],
        [ 0.,  0.,  8.]]])

In [40]:
float_dist[0][20]/100

0.3402154303084316

In [33]:
len(market_locations)

29